In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
from kaggle.api.kaggle_api_extended import KaggleApi
from collections import defaultdict

In [2]:
api = KaggleApi()
api.authenticate()

dataset_dir = Path("./dataset")

if dataset_dir.exists() and len(list(dataset_dir.glob("*.csv"))) == 12:
    print(f"Data already downloaded.")
else:
    api.dataset_download_files(
        'agungpambudi/network-malware-detection-connection-analysis',
        path='dataset',
        unzip=True
    )

files = sorted(dataset_dir.iterdir())

Data already downloaded.


In [3]:
def find_csv_delimiter(file_path, max_lines=5):
    with open(file_path, 'r', newline='') as f:
        sample = ''.join([f.readline() for _ in range(max_lines)])

    delimiters = [',', ';', '\t', '|']
    return max(delimiters, key=lambda d: sample.count(d))

In [4]:
def chunk_dataset(files, output_dir, chunk_size=500_000):

    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True, parents=True)

    for file_idx, file_path in enumerate(files):

        delimiter = find_csv_delimiter(file_path)
        print(f"[{file_idx + 1}/{len(files)}] Chunking:", file_path.name)

        reader = pd.read_csv(file_path, delimiter=delimiter, chunksize=chunk_size)

        for i, chunk in tqdm(enumerate(reader)):
            out_file = output_dir / f"{file_path.stem}_chunk{i}.csv"
            chunk.to_csv(out_file, index=False)

In [5]:
# chunk_dataset(files, "dataset_chunked_raw")

In [6]:
def accumulate_chunk_stats(files):

    total_rows = 0
    non_null_counts = defaultdict(int)
    dtypes_seen = defaultdict(set)

    for file_path in tqdm(files):

        for chunk in pd.read_csv(file_path, chunksize=100_000):

            rows = len(chunk)
            total_rows += rows

            # Non-null accumulation
            nn = chunk.notna().sum()

            for col, count in nn.items():
                non_null_counts[col] += int(count)

            # Track dtype drift (VERY useful)
            for col, dtype in chunk.dtypes.items():
                dtypes_seen[col].add(str(dtype))

    return total_rows, non_null_counts, dtypes_seen

In [7]:
files = list(Path("dataset_chunked_raw").iterdir())

total_rows, non_null_counts, dtypes_seen = accumulate_chunk_stats(files)

  5%|▌         | 3/59 [00:04<01:24,  1.50s/it]/tmp/ipykernel_57279/1566637975.py:9: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=100_000):
/tmp/ipykernel_57279/1566637975.py:9: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=100_000):
/tmp/ipykernel_57279/1566637975.py:9: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=100_000):
  7%|▋         | 4/59 [00:06<01:23,  1.51s/it]/tmp/ipykernel_57279/1566637975.py:9: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=100_000):
  8%|▊         | 5/59 [00:07<01:18,  1.45s/it]/tmp/ipykernel_57279/1566637975.py:9: DtypeWarning: Columns (22) have mixe

In [8]:
print("\nTOTAL ROWS:", total_rows)

print("\nCOLUMN STATS:")
for col in sorted(non_null_counts):

    nn = non_null_counts[col]
    nulls = total_rows - nn
    pct_null = (nulls / total_rows) * 100

    print(f"{col:20} non-null={nn:<10} null={nulls:<10} null%={pct_null:.2f}")

print("\nDTYPE DRIFT:")
for col, dtypes in dtypes_seen.items():
    if len(dtypes) > 1:
        print(col, "→", dtypes)


TOTAL ROWS: 25011003

COLUMN STATS:
conn_state           non-null=25011003   null=0          null%=0.00
detailed-label       non-null=15837049   null=9173954    null%=36.68
duration             non-null=25011003   null=0          null%=0.00
history              non-null=25011003   null=0          null%=0.00
id.orig_h            non-null=25011003   null=0          null%=0.00
id.orig_p            non-null=25011003   null=0          null%=0.00
id.resp_h            non-null=25011003   null=0          null%=0.00
id.resp_p            non-null=25011003   null=0          null%=0.00
label                non-null=25011003   null=0          null%=0.00
local_orig           non-null=25011003   null=0          null%=0.00
local_resp           non-null=25011003   null=0          null%=0.00
missed_bytes         non-null=25011003   null=0          null%=0.00
orig_bytes           non-null=25011003   null=0          null%=0.00
orig_ip_bytes        non-null=25011003   null=0          null%=0.00
orig_pkts 

In [9]:
def accumulate_unique_values(
    files,
    chunksize=100_000,
    max_track_uniques=5000,
):

    uniques = defaultdict(set)
    dropped_cols = set()

    for file_path in tqdm(files):

        for chunk in pd.read_csv(file_path, chunksize=chunksize):

            for col in chunk.columns:

                if col in dropped_cols:
                    continue

                values = chunk[col].dropna()
                if values.empty:
                    continue

                vals = values.astype(str).unique()
                uniques[col].update(vals)

                if len(uniques[col]) > max_track_uniques:
                    print(f"Dropping {col} (too many uniques)")
                    del uniques[col]
                    dropped_cols.add(col)

    return uniques, dropped_cols


def print_unique_summary(uniques, dropped_cols, max_display=20):

    print("\nEXACT UNIQUE SUMMARY")

    for col in sorted(uniques):

        vals = list(uniques[col])

        print(f"\nColumn: {col}")
        print(f"Exact unique count: {len(vals)}")

        if len(vals) <= max_display:
            print("Values:", vals)
        else:
            print("Sample values:", vals[:max_display])

    if dropped_cols:
        print("\nSkipped high-cardinality columns:")
        for col in sorted(dropped_cols):
            print(" -", col)

In [10]:
uniques, dropped_cols = accumulate_unique_values(files)
print_unique_summary(uniques, dropped_cols)

  0%|          | 0/59 [00:00<?, ?it/s]

Dropping ts (too many uniques)
Dropping uid (too many uniques)
Dropping id.orig_p (too many uniques)
Dropping id.resp_h (too many uniques)
Dropping duration (too many uniques)


  5%|▌         | 3/59 [00:06<02:09,  2.31s/it]/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):
/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):
/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):
  7%|▋         | 4/59 [00:09<02:04,  2.26s/it]/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):
  8%|▊         | 5/59 [00:11<02:02,  2.26s/it]/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (

Dropping id.resp_p (too many uniques)


 34%|███▍      | 20/59 [00:35<01:04,  1.67s/it]/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):
/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):
/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):
 37%|███▋      | 22/59 [00:37<00:50,  1.37s/it]/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):
/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (22) have mixed types. Specify dtype option o

Dropping orig_bytes (too many uniques)


 59%|█████▉    | 35/59 [01:02<00:47,  2.00s/it]/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):
 68%|██████▊   | 40/59 [01:13<00:42,  2.24s/it]/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):
/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):
 69%|██████▉   | 41/59 [01:16<00:40,  2.26s/it]

Dropping id.orig_h (too many uniques)


 76%|███████▋  | 45/59 [01:21<00:21,  1.57s/it]/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):
 78%|███████▊  | 46/59 [01:23<00:21,  1.65s/it]/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):
 90%|████████▉ | 53/59 [01:36<00:11,  1.96s/it]/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):
/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):
/tmp/ipykernel_57279/1980576605.py:12: DtypeWarning: Column


EXACT UNIQUE SUMMARY

Column: conn_state
Exact unique count: 13
Values: ['S3', 'RSTR', 'RSTO', 'S2', 'S0', 'RSTRH', 'OTH', 'SF', 'SH', 'S1', 'REJ', 'SHR', 'RSTOS0']

Column: detailed-label
Exact unique count: 7
Values: ['Torii', 'PartOfAHorizontalPortScan', 'C&C', '-', 'FileDownload', 'Attack', 'HeartBeat']

Column: history
Exact unique count: 264
Sample values: ['ShAdDaFrR', 'ShADadfrr', 'ShAadDFR', 'ShADfFrr', 'ShAdDaf', 'R', 'Ffa', 'ShArR', 'ShA', 'ShAfF', 'ShAdfDF', 'ShAdDaFfR', 'DAd', 'DaFfA', 'ShAdDatFf', 'ShAdFaf', 'ShAdDaFRf', 'SahAdDF', 'Dr', 'ShAfdtDFr']

Column: label
Exact unique count: 7
Values: ['Malicious', 'Malicious   PartOfAHorizontalPortScan', 'Benign', 'Malicious   C&C', 'Malicious   Attack', 'Malicious   DDoS', 'Malicious   FileDownload']

Column: local_orig
Exact unique count: 1
Values: ['-']

Column: local_resp
Exact unique count: 1
Values: ['-']

Column: missed_bytes
Exact unique count: 24
Sample values: ['8688', '0.0', '5792', '2920', '8160', '20272', '17376',

In [11]:
def process_chunks(processing_pipeline, input_dir, output_dir, kwargs={}):

    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)

    for file_path in tqdm(sorted(input_dir.iterdir())):

        df = pd.read_csv(file_path)
        df_processed = processing_pipeline(df, **kwargs)

        out_file = output_dir / file_path.name
        df_processed.to_csv(out_file, index=False)

In [12]:
from preprocessing_scripts import run_through_pipeline

process_chunks(run_through_pipeline, "./dataset_chunked_raw", "./dataset_chunked_processed")

  0%|          | 0/59 [00:00<?, ?it/s]

[unify_numeric_columns] Column 'id.orig_p' not in DataFrame. Skipping...
[unify_numeric_columns] Column 'id.resp_p' not in DataFrame. Skipping...
[map_features] Mapping for feature 'conn_state' contains labels not present in data: {'S3', 'SHR'}
[map_features] Mapping for feature 'label' contains labels not present in data: {'Malicious   PartOfAHorizontalPortScan', 'Malicious   C&C', 'Malicious   Attack', 'Malicious   DDoS', 'Malicious   FileDownload'}


  2%|▏         | 1/59 [00:09<09:29,  9.83s/it]

[unify_numeric_columns] Column 'id.orig_p' not in DataFrame. Skipping...
[unify_numeric_columns] Column 'id.resp_p' not in DataFrame. Skipping...
[map_features] Mapping for feature 'conn_state' contains labels not present in data: {'S3', 'SHR'}
[map_features] Mapping for feature 'label' contains labels not present in data: {'Malicious   PartOfAHorizontalPortScan', 'Malicious   C&C', 'Malicious   Attack', 'Malicious   DDoS', 'Malicious   FileDownload'}


  8%|▊         | 5/59 [00:18<01:59,  2.21s/it]

...

[unify_numeric_columns] Column 'id.orig_p' not in DataFrame. Skipping...
[unify_numeric_columns] Column 'id.resp_p' not in DataFrame. Skipping...
[map_features] Mapping for feature 'conn_state' contains labels not present in data: {'S3', 'RSTRH', 'SH', 'S1', 'SHR', 'RSTOS0'}
[map_features] Mapping for feature 'label' contains labels not present in data: {'Malicious   PartOfAHorizontalPortScan', 'Malicious   C&C', 'Malicious   Attack', 'Malicious   DDoS', 'Malicious   FileDownload'}


 98%|█████████▊| 58/59 [07:47<00:07,  7.38s/it]

[unify_numeric_columns] Column 'id.orig_p' not in DataFrame. Skipping...
[unify_numeric_columns] Column 'id.resp_p' not in DataFrame. Skipping...
[map_features] Mapping for feature 'conn_state' contains labels not present in data: {'S3', 'S2', 'RSTRH', 'OTH', 'SH', 'S1', 'SHR', 'RSTOS0'}
[map_features] Mapping for feature 'proto' contains labels not present in data: {'icmp'}
[map_features] Mapping for feature 'label' contains labels not present in data: {'Malicious   PartOfAHorizontalPortScan', 'Malicious   C&C', 'Malicious   Attack', 'Malicious   DDoS', 'Malicious   FileDownload'}


100%|██████████| 59/59 [07:55<00:00,  8.06s/it]

[unify_numeric_columns] Column 'id.orig_p' not in DataFrame. Skipping...
[unify_numeric_columns] Column 'id.resp_p' not in DataFrame. Skipping...
[map_features] Mapping for feature 'conn_state' contains labels not present in data: {'S3', 'RSTR', 'RSTRH', 'SH', 'S1', 'SHR', 'RSTOS0'}
[map_features] Mapping for feature 'label' contains labels not present in data: {'Malicious   PartOfAHorizontalPortScan', 'Malicious   C&C', 'Malicious   Attack', 'Malicious   DDoS', 'Malicious   FileDownload'}


In [13]:
files = sorted(list(Path("./dataset_chunked_processed").iterdir()))

uniques, dropped_cols = accumulate_unique_values(files)
print_unique_summary(uniques, dropped_cols)

  0%|          | 0/59 [00:00<?, ?it/s]

Dropping duration_log (too many uniques)


 17%|█▋        | 10/59 [00:08<00:58,  1.19s/it]

Dropping orig_bytes_log (too many uniques)


100%|██████████| 59/59 [00:59<00:00,  1.01s/it]


EXACT UNIQUE SUMMARY

Column: conn_state
Exact unique count: 13
Values: ['10', '9', '11', '5', '4', '0', '7', '3', '8', '12', '1', '2', '6']

Column: hist_has_data
Exact unique count: 2
Values: ['1', '0']

Column: hist_has_fin
Exact unique count: 2
Values: ['1', '0']

Column: hist_has_rst
Exact unique count: 2
Values: ['1', '0']

Column: hist_has_syn
Exact unique count: 2
Values: ['1', '0']

Column: label
Exact unique count: 7
Values: ['5', '4', '0', '3', '1', '2', '6']

Column: orig_ephemeral
Exact unique count: 2
Values: ['1', '0']

Column: orig_ip_bytes_log
Exact unique count: 1847
Sample values: ['7.237778191923443', '6.455198563340122', '7.339537695407674', '6.86171134048073', '6.974478911025045', '7.597396320212795', '7.652545692693921', '6.447305862541213', '6.843749949006225', '12.002737848944353', '7.950854857719988', '7.390798521735676', '7.183111701743281', '6.80903930604298', '21.30064936021981', '7.139660335964919', '7.989899374942939', '8.281723990411392', '7.53529670244

In [18]:
total_rows, non_null_counts, dtypes_seen = accumulate_chunk_stats(files)
print("\nTOTAL ROWS:", total_rows)

print("\nCOLUMN STATS:")
for col in sorted(non_null_counts):

    nn = non_null_counts[col]
    nulls = total_rows - nn
    pct_null = (nulls / total_rows) * 100

    print(f"{col:20} non-null={nn:<10} null={nulls:<10} null%={pct_null:.2f}")

print("\nDTYPE DRIFT:")
for col, dtypes in dtypes_seen.items():
    if len(dtypes) > 1:
        print(col, "→", dtypes)

100%|██████████| 59/59 [00:09<00:00,  6.06it/s]


TOTAL ROWS: 9738930

COLUMN STATS:
conn_state           non-null=9738930    null=0          null%=0.00
duration_log         non-null=9738930    null=0          null%=0.00
hist_has_data        non-null=9738930    null=0          null%=0.00
hist_has_fin         non-null=9738930    null=0          null%=0.00
hist_has_rst         non-null=9738930    null=0          null%=0.00
hist_has_syn         non-null=9738930    null=0          null%=0.00
label                non-null=9738930    null=0          null%=0.00
orig_bytes_log       non-null=9738930    null=0          null%=0.00
orig_ephemeral       non-null=9738930    null=0          null%=0.00
orig_ip_bytes_log    non-null=9738930    null=0          null%=0.00
orig_is_private      non-null=9738930    null=0          null%=0.00
orig_pkts_log        non-null=9738930    null=0          null%=0.00
proto                non-null=9738930    null=0          null%=0.00
resp_bytes_log       non-null=9738930    null=0          null%=0.00
resp_ip_byte

In [14]:
import joblib
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [ ]:
columns_to_scale = [
    "duration_log",
    "orig_pkts_log",
    "resp_pkts_log",
    "orig_bytes_log",
    "resp_bytes_log",
    "orig_ip_bytes_log",
    "resp_ip_bytes_log"
]

files = sorted(list(Path("./dataset_chunked_processed").iterdir()))

for file_path in tqdm(files):
    for chunk in pd.read_csv(file_path, chunksize=100_000):

        X = chunk[columns_to_scale]

        try:
            scaler.partial_fit(X)
        except ValueError:
            continue

save_path = Path("./model/scaler.pkl")
save_path.parent.mkdir(exist_ok=True, parents=True)
joblib.dump(scaler, save_path)

100%|██████████| 59/59 [00:10<00:00,  5.44it/s]


['utils/scaler.pkl']

In [20]:
SPLIT_RATIOS = {
    "train": 0.20,
    "test": 0.10,
    "producer": 0.70
}
rng = np.random.default_rng(seed=42)

In [21]:
def stratified_chunk_split(
    df,
    label_col,
    ratios,
    rng
):
    parts = []

    for label, group in df.groupby(label_col):

        n = len(group)
        if n == 0:
            continue

        group = group.sample(frac=1, random_state=rng.integers(1e9))

        n_train = int(n * ratios["train"])
        n_test = int(n * ratios["test"])

        train = group.iloc[:n_train]
        test = group.iloc[n_train:n_train + n_test]
        producer = group.iloc[n_train + n_test:]

        parts.append(("train", train))
        parts.append(("test", test))
        parts.append(("producer", producer))

    return parts

In [26]:
def process_and_split_files(files, output_dir, chunksize=500_000):

    output_dir = Path(output_dir)
    (output_dir / "train").mkdir(parents=True, exist_ok=True)
    (output_dir / "test").mkdir(parents=True, exist_ok=True)
    (output_dir / "producer").mkdir(parents=True, exist_ok=True)

    rng = np.random.default_rng(42)

    for file_path in tqdm(files):

        for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunksize)):

            splits = stratified_chunk_split(
                chunk,
                label_col="label",
                ratios=SPLIT_RATIOS,
                rng=rng
            )

            for split_name, split_df in splits:

                if split_df.empty:
                    continue

                split_df[columns_to_scale] = scaler.transform(
                    split_df[columns_to_scale]
                )

                out_file = output_dir / split_name / f"{file_path.stem}.csv"

                split_df.to_csv(
                    out_file,
                    mode="a",
                    header=not out_file.exists(),
                    index=False
                )

In [27]:
process_and_split_files(files, "./dataset_split")

100%|██████████| 59/59 [02:04<00:00,  2.10s/it]


In [30]:
files_t = sorted(list(Path("./dataset_split/producer").iterdir()))
total_rows, non_null_counts, dtypes_seen = accumulate_chunk_stats(files_t)
print("\nTOTAL ROWS:", total_rows)

print("\nCOLUMN STATS:")
for col in sorted(non_null_counts):

    nn = non_null_counts[col]
    nulls = total_rows - nn
    pct_null = (nulls / total_rows) * 100

    print(f"{col:20} non-null={nn:<10} null={nulls:<10} null%={pct_null:.2f}")

print("\nDTYPE DRIFT:")
for col, dtypes in dtypes_seen.items():
    if len(dtypes) > 1:
        print(col, "→", dtypes)

100%|██████████| 53/53 [00:11<00:00,  4.80it/s]


TOTAL ROWS: 7118035

COLUMN STATS:
conn_state           non-null=7118035    null=0          null%=0.00
duration_log         non-null=7118035    null=0          null%=0.00
hist_has_data        non-null=7118035    null=0          null%=0.00
hist_has_fin         non-null=7118035    null=0          null%=0.00
hist_has_rst         non-null=7118035    null=0          null%=0.00
hist_has_syn         non-null=7118035    null=0          null%=0.00
label                non-null=7118035    null=0          null%=0.00
orig_bytes_log       non-null=7118035    null=0          null%=0.00
orig_ephemeral       non-null=7118035    null=0          null%=0.00
orig_ip_bytes_log    non-null=7118035    null=0          null%=0.00
orig_is_private      non-null=7118035    null=0          null%=0.00
orig_pkts_log        non-null=7118035    null=0          null%=0.00
proto                non-null=7118035    null=0          null%=0.00
resp_bytes_log       non-null=7118035    null=0          null%=0.00
resp_ip_byte